In [1]:
from darksky import forecast
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [2]:
flights = pd.read_csv('data/basic_features.csv', index_col = 0)
print flights.shape
flights.head()

(246743, 9)


,ARRIVAL_DELAY,DAY,MONTH,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAIL_NUMBER
63,1.0,1,1,4,UA,ORD,IAH,5,N30401
93,1.0,1,1,4,UA,ORD,DEN,5,N66837
100,0.0,1,1,4,EV,ORD,DTW,5,N132EV
422,0.0,1,1,4,UA,ORD,MCO,6,N422UA
427,0.0,1,1,4,OO,ORD,MBS,6,N979SW


In [3]:
col_names = flights.columns
weather_cols = ['TEMPERATURE', 'ICON', 'HUMIDITY', 'SUMMARY', 'WIND_SPEED', 'VISIBILITY', 'PRECIP_INTENSITY', 'PRECIP_PROB']
new_col_names = list(col_names) + list(weather_cols)
print new_col_names

['ARRIVAL_DELAY', 'DAY', 'MONTH', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'TAIL_NUMBER', 'TEMPERATURE', 'ICON', 'HUMIDITY', 'SUMMARY', 'WIND_SPEED', 'VISIBILITY', 'PRECIP_INTENSITY', 'PRECIP_PROB']


In [4]:
flights = flights.reindex(columns = new_col_names)
flights = flights.reset_index(drop=True)
flights['ICON'] = flights.ICON.astype(str)
flights['SUMMARY'] = flights.SUMMARY.astype(str)
print flights.shape
flights.head()

(246743, 17)


,ARRIVAL_DELAY,DAY,MONTH,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAIL_NUMBER,TEMPERATURE,ICON,HUMIDITY,SUMMARY,WIND_SPEED,VISIBILITY,PRECIP_INTENSITY,PRECIP_PROB
0,1.0,1,1,4,UA,ORD,IAH,5,N30401,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN
1,1.0,1,1,4,UA,ORD,DEN,5,N66837,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN
2,0.0,1,1,4,EV,ORD,DTW,5,N132EV,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN
3,0.0,1,1,4,UA,ORD,MCO,6,N422UA,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN
4,0.0,1,1,4,OO,ORD,MBS,6,N979SW,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN


In [5]:
api_key = '6a9b6da88446b37ee25e51a07eb5f1e0'
lat = 41.977
longt = -87.837
year = '2015'

# minutes, seconds, and time zone for query
ending = ':00:00-0600'

In [6]:
data_cols = flights.columns

# get relevant column indexes
day_col = data_cols.get_loc('DAY')
month_col = data_cols.get_loc('MONTH')
hour_col = data_cols.get_loc('SCHEDULED_DEPARTURE')
temp_col = data_cols.get_loc('TEMPERATURE')
icon_col = data_cols.get_loc('ICON')
humidity_col = data_cols.get_loc('HUMIDITY')
summary_col = data_cols.get_loc('SUMMARY')
wind_col = data_cols.get_loc('WIND_SPEED')
visibility_col = data_cols.get_loc('VISIBILITY')
precip_col = data_cols.get_loc('PRECIP_INTENSITY')
precip_prob_col = data_cols.get_loc('PRECIP_PROB')

In [34]:
for i, row in flights.iterrows():
    if i % 5000 == 0:
        print ('done ' + str(i))
        flights.to_csv('data/weather_features.csv')
    
    #check if already queried weather for that flight
    if str(flights.iat[i, temp_col]) != 'nan':
        continue
    
    #extract data needed for Dark Sky Weather API Request
    day = str(flights.iat[i,day_col])
    month = str(flights.iat[i,month_col])
    hour = str(flights.iat[i,hour_col])
    if len(day) == 1:
        day = '0'+day
    if len(month) == 1:
        month = '0'+month
    if len(hour) == 1:
        hour = '0'+hour
    
    # make request
    t = year + '-' + month + '-' + day + 'T' + hour + ending
    request = forecast(api_key, lat, longt, time = t)
    
    # update dataframe
    flights.iat[i,temp_col] = float(request['currently']['temperature'])
    flights.iat[i,icon_col] = str(request['currently']['icon'])
    flights.iat[i,humidity_col] = float(request['currently']['humidity'])
    flights.iat[i,summary_col] = str(request['currently']['summary'])
    flights.iat[i,wind_col] = float(request['currently']['windSpeed'])
    flights.iat[i,visibility_col] = float(request['currently']['visibility'])
    flights.iat[i,precip_col] = float(request['currently']['precipIntensity'])
    flights.iat[i,precip_prob_col] = float(request['currently']['precipProbability'])

done 0
done 5000
done 10000
done 15000
done 20000
done 25000
done 30000
done 35000
done 40000
done 45000
done 50000
done 55000
done 60000
done 65000


KeyboardInterrupt: 

In [8]:
flights.head()

,ARRIVAL_DELAY,DAY,MONTH,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAIL_NUMBER,TEMPERATURE,ICON,HUMIDITY,SUMMARY,WIND_SPEED,VISIBILITY,PRECIP_INTENSITY,PRECIP_PROB
0,1.0,1,1,4,UA,ORD,IAH,5,N30401,15.43,clear-night,0.65,Clear,14.19,10.0,0.0,0.0
1,1.0,1,1,4,UA,ORD,DEN,5,N66837,15.43,clear-night,0.65,Clear,14.19,10.0,0.0,0.0
2,0.0,1,1,4,EV,ORD,DTW,5,N132EV,15.43,clear-night,0.65,Clear,14.19,10.0,0.0,0.0
3,0.0,1,1,4,UA,ORD,MCO,6,N422UA,15.76,clear-night,0.64,Clear,15.01,10.0,0.0,0.0
4,0.0,1,1,4,OO,ORD,MBS,6,N979SW,15.76,clear-night,0.64,Clear,15.01,10.0,0.0,0.0


In [35]:
flights.to_csv('data/weather_features.csv')

In [32]:
i = 0
if str(flights.iat[150000, temp_col]) != 'nan':
    print 'filled'
print flights.iat[150000,temp_col]

nan
